In [1]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
# from fastai.column_data import *
# from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.torch_imports import save_model, load_model

In [4]:
from models import *
from utils import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
# Experiment Code

In [6]:
EXP_CODE = "SOTA_MASK"

In [7]:
! ls -f KITTI/ | grep .csv

validation.csv
validation_192_640.csv
training.csv
validation_192_640_pre.csv
training_192_640_pre.csv
training_192_640.csv


In [8]:
train, val = pd.read_csv('./KITTI/training_192_640_pre.csv'), pd.read_csv('./KITTI/validation_192_640_pre.csv')
train, val = shuffle(train), shuffle(val);

In [9]:
train.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_26/2011_09_26_drive_0056_sync/image_02...,2011_09_26/2011_09_26_drive_0056_sync/image_02...,2011_09_26/2011_09_26_drive_0056_sync/image_02...,371.806866,369.427302,314.104631,88.501248
1,2011_10_03/2011_10_03_drive_0027_sync/image_02...,2011_10_03/2011_10_03_drive_0027_sync/image_02...,2011_10_03/2011_10_03_drive_0027_sync/image_02...,370.424992,368.054272,312.885179,94.830438
2,2011_10_03/2011_10_03_drive_0027_sync/image_03...,2011_10_03/2011_10_03_drive_0027_sync/image_03...,2011_10_03/2011_10_03_drive_0027_sync/image_03...,370.424992,368.054272,312.885179,94.830438
3,2011_10_03/2011_10_03_drive_0027_sync/image_02...,2011_10_03/2011_10_03_drive_0027_sync/image_02...,2011_10_03/2011_10_03_drive_0027_sync/image_02...,370.424992,368.054272,312.885179,94.830438
4,2011_10_03/2011_10_03_drive_0027_sync/image_02...,2011_10_03/2011_10_03_drive_0027_sync/image_02...,2011_10_03/2011_10_03_drive_0027_sync/image_02...,370.424992,368.054272,312.885179,94.830438


In [10]:
val.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_26/2011_09_26_drive_0029_sync/image_02...,2011_09_26/2011_09_26_drive_0029_sync/image_02...,2011_09_26/2011_09_26_drive_0029_sync/image_02...,371.806866,369.427302,314.104631,88.501248
1,2011_09_30/2011_09_30_drive_0034_sync/image_03...,2011_09_30/2011_09_30_drive_0034_sync/image_03...,2011_09_30/2011_09_30_drive_0034_sync/image_03...,364.362615,362.030694,310.151266,93.752525
2,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,370.424992,368.054272,312.885179,94.830438
3,2011_09_26/2011_09_26_drive_0086_sync/image_03...,2011_09_26/2011_09_26_drive_0086_sync/image_03...,2011_09_26/2011_09_26_drive_0086_sync/image_03...,371.806866,369.427302,314.104631,88.501248
4,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,370.424992,368.054272,312.885179,94.830438


In [11]:
# make sure nothing sneak into the val set
a = "0016_sync"
for i, row in train.iterrows():
    if a in row.t0:
        print(row.t0)

In [12]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [13]:
trn_camera.head()

,fx,fy,cx,cy
0,371.806866,369.427302,314.104631,88.501248
1,370.424992,368.054272,312.885179,94.830438
2,370.424992,368.054272,312.885179,94.830438
3,370.424992,368.054272,312.885179,94.830438
4,370.424992,368.054272,312.885179,94.830438


In [14]:
arch = f
bs = 8
PATH = 'Fastai_TRN'
DPATH = "KITTI/"
verbose = 100

In [15]:
f

<function torchvision.models.resnet.resnet18(pretrained=False, **kwargs)>

In [16]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [17]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [18]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)

In [19]:
tfms = (trn_tfms, val_tfms)

MD1 = get_MD(trn0, val0, tfms, bs, DPATH, PATH)
MD2 = get_MD(trn1, val1, tfms, bs, DPATH, PATH)
MD3 = get_MD(trn2, val2, tfms, bs, DPATH, PATH)
MDcam = get_cam(trn_camera, val_camera, bs)

In [20]:
imgnet_mean, imgnet_std = torch.from_numpy(stats[0]).float(), torch.from_numpy(stats[1]).float()
imgnet_mean, imgnet_std = imgnet_mean.view(1,3,1,1), imgnet_std.view(1,3,1,1)
denorm = denormer(imgnet_mean, imgnet_std)

In [21]:
def train(m, opt, MD1, MD2, MD3, MDcam, denorm):
    collector = LossCollect(6)
    
    m.train()
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []  
    
    #for i in range(2): # just for testing
    for i in range(len(MD1.trn_ds)//bs-len(losses)):
        t1 = time.time()
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])
        
        d1, d2, d3, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 
        #pdb.set_trace()
        loss, details = l(d1, d2, d3, trans, rotation, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        t2 = time.time()
        
        collector.collect(
            loss.data[0],
            details[0].data[0],
            details[1].data[0],
            details[2].data[0],
            details[3].data[0],
            t2-t1
            )
        
        # total appr smooth ssim l1
        if i%verbose == 0: print("{:.4f}, {:.4f}, {:.4E}, {:.4f}, {:.4f}, {:.2f}s/batch".format(*collector.show()))
    return collector.collections[0]  #return the total losses

In [22]:
def evaluate(m, MD1, MD2, MD3, MDcam, folder):
    
    DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
    m.eval()
    for i in range(6): 
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
        d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1, volatile=True), denorm(imgs2, volatile=True), denorm(imgs3, volatile=True) 
        #pdb.set_trace()
        d2 = d2s[0]
        # d2 = F.upsample(input=d2, scale_factor=2**di, mode='bilinear')
        cx12, cy12, dm12 = l.appr.offset.forward(trans=trans[:,0], rotation=rotation[:,0], inv_depth = d2, camera = cam)
        #pdb.set_trace()

        x12, ivm12 = l.appr.sampler.forward(imgs1, cx12, cy12)
        ivm12.volatile = True

        del dm12 
        del ivm12
        
        imgs2 = tonp(imgs2.permute(0,2,3,1))
        x12 = tonp(x12.permute(0,2,3,1))
        d2 = tonp(d2)
        
        for j in range(bs):
            save_res(imgs2[j], x12[j], d2[j][0], folder/"{}{}.png".format(i,j))

In [23]:
def fit(epoch, m, opt, MD1, MD2, MD3, MDcam):
    for i in range(epoch):
        print("--------------------epoch {} start:----------------------".format(i))
        losses = train(m, opt, MD1, MD2, MD3, MDcam)
        losses= [ str(loss) for loss in losses ]
        folder = Path("./tmp")
        folder.mkdir(exist_ok=True)
        file = folder / 'epoch{}.log'.format(i)
        with file.open('w') as f:
            f.write("\n".join(losses))
        save_model(m, str(folder / "epoch{}.M".format(i)))

In [24]:
expfolder = Path('./experiment/')/EXP_CODE
expfolder.mkdir(parents=True, exist_ok=True)

In [25]:
m = TriDepth(get_base(f, cut), 1, setting=[False, False, False]).cuda()
l = Loss(smooth_scale=0.001, appr_scale=0.85, warp_setting=[True, True, True, True], is_per_pixel_min=True, is_mask=True).cuda()
opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)


# Run this if necessary
from fastai.torch_imports import save_model, load_model
path = expfolder/'epoch_2'/'model.M'
load_model(m, str(path))

In [ ]:
start = 0 # should be 0 if normal 
epoch = 8
#     set_trainable(m.depth.rn, False)
for i in range(start,epoch):    
    thisfolder = expfolder/"epoch_{}".format(i+1)
    thisfolder.mkdir(exist_ok=True)

    print("Epoch {} start".format(i+1))
    print("------------------training model--------------------")
    losses = train(m, opt, MD1, MD2, MD3, MDcam, denorm)
    print("------------------saving model-------------------")
    save_model(m, thisfolder/"model.M")
    print("------------------writting log-------------------")
    logger = thisfolder/"log"
    losses = [ str(loss) for loss in losses]
    with logger.open('w') as f: f.write("\n".join(losses))
    print("---------------------predicting------------------")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

Epoch 1 start
------------------training model--------------------
0.1623, 0.1623, 3.8872E-05, 0.1623, 0.1623, 14.59s/batch
0.1108, 0.1108, 2.4205E-05, 0.1108, 0.1108, 1.46s/batch
0.1077, 0.1077, 2.2157E-05, 0.1077, 0.1077, 1.48s/batch
0.1007, 0.1006, 2.1737E-05, 0.1006, 0.1006, 1.14s/batch
0.0935, 0.0935, 1.9042E-05, 0.0935, 0.0935, 1.57s/batch
0.0810, 0.0810, 1.7747E-05, 0.0810, 0.0810, 1.60s/batch
0.0870, 0.0870, 1.5435E-05, 0.0870, 0.0870, 1.47s/batch
0.0805, 0.0805, 1.5344E-05, 0.0805, 0.0805, 1.61s/batch
0.0850, 0.0850, 1.5512E-05, 0.0850, 0.0850, 2.52s/batch
0.0823, 0.0823, 1.5984E-05, 0.0823, 0.0823, 1.14s/batch
0.0795, 0.0795, 1.4904E-05, 0.0795, 0.0795, 1.14s/batch
0.0813, 0.0813, 1.4268E-05, 0.0813, 0.0813, 1.15s/batch
0.0733, 0.0732, 1.4218E-05, 0.0732, 0.0732, 1.15s/batch
0.0792, 0.0792, 1.3843E-05, 0.0792, 0.0792, 1.50s/batch
0.0717, 0.0717, 1.2899E-05, 0.0717, 0.0717, 1.47s/batch
0.0824, 0.0823, 1.3370E-05, 0.0823, 0.0823, 1.46s/batch
0.0802, 0.0802, 1.2892E-05, 0.0802, 

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 2 start
------------------training model--------------------
0.0866, 0.0866, 1.2752E-05, 0.0866, 0.0866, 9.62s/batch
0.0614, 0.0614, 1.2341E-05, 0.0614, 0.0614, 1.56s/batch
0.0682, 0.0682, 1.2641E-05, 0.0682, 0.0682, 1.46s/batch
0.0681, 0.0681, 1.2126E-05, 0.0681, 0.0681, 1.58s/batch
0.0671, 0.0671, 1.2199E-05, 0.0671, 0.0671, 1.53s/batch
0.0616, 0.0615, 1.2446E-05, 0.0615, 0.0615, 1.58s/batch
0.0713, 0.0713, 1.2190E-05, 0.0713, 0.0713, 1.23s/batch
0.0668, 0.0668, 1.2861E-05, 0.0668, 0.0668, 1.56s/batch
0.0732, 0.0732, 1.1813E-05, 0.0732, 0.0732, 2.96s/batch
0.0715, 0.0715, 1.2827E-05, 0.0715, 0.0715, 1.58s/batch
0.0682, 0.0682, 1.2355E-05, 0.0682, 0.0682, 1.45s/batch
0.0711, 0.0711, 1.2050E-05, 0.0711, 0.0711, 1.43s/batch
0.0633, 0.0633, 1.1731E-05, 0.0633, 0.0633, 1.55s/batch
0.0721, 0.0721, 1.2051E-05, 0.0721, 0.0721, 1.55s/batch
0.0641, 0.0641, 1.2038E-05, 0.0641, 0.0641, 1.51s/batch
0.0711, 0.0711, 1.1772E-05, 0.0711, 0.0711, 1.38s/batch
0.0718, 0.0718, 1.2214E-05, 0.0718, 0

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 3 start
------------------training model--------------------
0.0703, 0.0703, 1.2985E-05, 0.0703, 0.0703, 8.70s/batch
0.0542, 0.0542, 1.1736E-05, 0.0542, 0.0542, 1.43s/batch
0.0596, 0.0596, 1.2402E-05, 0.0596, 0.0596, 1.52s/batch
0.0585, 0.0585, 1.2275E-05, 0.0585, 0.0585, 1.59s/batch
0.0567, 0.0567, 1.2493E-05, 0.0567, 0.0567, 1.52s/batch
0.0554, 0.0554, 1.2735E-05, 0.0554, 0.0554, 1.48s/batch
0.0587, 0.0587, 1.2194E-05, 0.0587, 0.0587, 1.56s/batch
0.0572, 0.0572, 1.2571E-05, 0.0572, 0.0572, 1.42s/batch
0.0602, 0.0602, 1.1834E-05, 0.0602, 0.0602, 3.19s/batch
0.0619, 0.0619, 1.2319E-05, 0.0619, 0.0619, 1.55s/batch
0.0591, 0.0591, 1.1707E-05, 0.0591, 0.0591, 1.52s/batch
0.0599, 0.0599, 1.1871E-05, 0.0599, 0.0599, 1.49s/batch
0.0545, 0.0545, 1.1873E-05, 0.0545, 0.0545, 1.49s/batch
0.0604, 0.0604, 1.1661E-05, 0.0604, 0.0604, 1.52s/batch
0.0559, 0.0558, 1.1938E-05, 0.0558, 0.0558, 1.43s/batch
0.0611, 0.0611, 1.1654E-05, 0.0611, 0.0611, 1.57s/batch
0.0600, 0.0600, 1.1431E-05, 0.0600, 0

...

Epoch 4 start
------------------training model--------------------
0.0651, 0.0650, 1.3467E-05, 0.0650, 0.0650, 6.66s/batch
0.0491, 0.0491, 1.1676E-05, 0.0491, 0.0491, 1.14s/batch
0.0538, 0.0538, 1.2164E-05, 0.0538, 0.0538, 1.14s/batch
0.0546, 0.0546, 1.1793E-05, 0.0546, 0.0546, 1.14s/batch
0.0522, 0.0522, 1.2313E-05, 0.0522, 0.0522, 1.15s/batch
0.0507, 0.0507, 1.2299E-05, 0.0507, 0.0507, 1.14s/batch
0.0531, 0.0531, 1.2163E-05, 0.0531, 0.0531, 1.14s/batch
0.0509, 0.0509, 1.2343E-05, 0.0509, 0.0509, 1.14s/batch
0.0551, 0.0551, 1.1435E-05, 0.0551, 0.0551, 2.05s/batch
0.0575, 0.0574, 1.2155E-05, 0.0574, 0.0574, 1.14s/batch
0.0533, 0.0533, 1.1706E-05, 0.0533, 0.0533, 1.15s/batch
0.0519, 0.0519, 1.1203E-05, 0.0519, 0.0519, 1.14s/batch
0.0487, 0.0487, 1.1797E-05, 0.0487, 0.0487, 1.16s/batch
0.0513, 0.0513, 1.1363E-05, 0.0513, 0.0513, 1.15s/batch
0.0501, 0.0501, 1.1848E-05, 0.0501, 0.0501, 1.13s/batch
0.0542, 0.0541, 1.1711E-05, 0.0541, 0.0541, 1.14s/batch
0.0518, 0.0518, 1.1650E-05, 0.0518, 0

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 5 start
------------------training model--------------------
0.0632, 0.0632, 1.3056E-05, 0.0632, 0.0632, 6.39s/batch
0.0455, 0.0455, 1.1647E-05, 0.0455, 0.0455, 1.15s/batch
0.0502, 0.0502, 1.2370E-05, 0.0502, 0.0502, 1.14s/batch
0.0540, 0.0540, 1.2029E-05, 0.0540, 0.0540, 1.14s/batch
0.0489, 0.0489, 1.2335E-05, 0.0489, 0.0489, 1.15s/batch
0.0467, 0.0467, 1.1839E-05, 0.0467, 0.0467, 1.14s/batch
0.0489, 0.0489, 1.2103E-05, 0.0489, 0.0489, 1.14s/batch
0.0477, 0.0477, 1.2104E-05, 0.0477, 0.0477, 1.14s/batch
0.0483, 0.0483, 1.2127E-05, 0.0483, 0.0483, 2.23s/batch
0.0580, 0.0580, 1.2177E-05, 0.0580, 0.0580, 1.15s/batch
0.0521, 0.0521, 1.1545E-05, 0.0521, 0.0521, 1.14s/batch
0.0485, 0.0485, 1.1665E-05, 0.0485, 0.0485, 1.14s/batch
0.0443, 0.0443, 1.1483E-05, 0.0443, 0.0443, 1.14s/batch
0.0478, 0.0478, 1.1597E-05, 0.0478, 0.0478, 1.14s/batch
0.0507, 0.0507, 1.2342E-05, 0.0507, 0.0507, 1.14s/batch
0.0526, 0.0526, 1.2031E-05, 0.0526, 0.0526, 1.15s/batch
0.0518, 0.0518, 1.1706E-05, 0.0518, 0

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Epoch 6 start
------------------training model--------------------
0.0544, 0.0544, 1.1940E-05, 0.0544, 0.0544, 5.70s/batch
0.0433, 0.0433, 1.1209E-05, 0.0433, 0.0433, 1.14s/batch
0.0489, 0.0489, 1.1358E-05, 0.0489, 0.0489, 1.14s/batch
0.0492, 0.0492, 1.1282E-05, 0.0492, 0.0492, 1.14s/batch
0.0482, 0.0482, 1.2490E-05, 0.0482, 0.0482, 1.16s/batch
0.0467, 0.0467, 1.1682E-05, 0.0467, 0.0467, 1.14s/batch
0.0473, 0.0473, 1.1956E-05, 0.0473, 0.0473, 1.14s/batch
0.0474, 0.0474, 1.1924E-05, 0.0474, 0.0474, 1.14s/batch
0.0444, 0.0444, 1.1488E-05, 0.0444, 0.0444, 1.97s/batch
0.0491, 0.0491, 1.2588E-05, 0.0491, 0.0491, 1.14s/batch
0.0495, 0.0495, 1.1701E-05, 0.0495, 0.0495, 1.16s/batch
0.0462, 0.0462, 1.1703E-05, 0.0462, 0.0462, 1.14s/batch
0.0446, 0.0446, 1.1276E-05, 0.0446, 0.0446, 1.14s/batch
0.0441, 0.0441, 1.1146E-05, 0.0441, 0.0441, 1.14s/batch
0.0457, 0.0456, 1.1938E-05, 0.0456, 0.0456, 1.14s/batch
0.0506, 0.0506, 1.1364E-05, 0.0506, 0.0506, 1.15s/batch
0.0477, 0.0477, 1.1228E-05, 0.0477, 0

In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
m.eval()


In [ ]:
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        

In [ ]:
d2s[0].min(), d2s[0].max()